In [2]:
import pandas as pd

# Replace 'your_file_path.csv' with the path of your uploaded CSV file

df = pd.read_csv('resources/fnew-2023-2.csv')
print(df.head(10))

                   dtm       f
0  2023-02-01 00:00:00  50.083
1  2023-02-01 00:00:01  50.073
2  2023-02-01 00:00:02  50.061
3  2023-02-01 00:00:03  50.050
4  2023-02-01 00:00:04  50.044
5  2023-02-01 00:00:05  50.040
6  2023-02-01 00:00:06  50.038
7  2023-02-01 00:00:07  50.040
8  2023-02-01 00:00:08  50.039
9  2023-02-01 00:00:09  50.035


In [2]:
import cv2
import numpy as np
import random
import pandas as pd

def extract_random_segment(video_path, segment_length_min=10):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise IOError("Cannot open video file")

    fps = cap.get(cv2.CAP_PROP_FPS)  # Frame rate of the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames in the video
    total_duration = total_frames / fps  # Total duration of the video in seconds
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    codec = int(cap.get(cv2.CAP_PROP_FOURCC))  # Codec of the video
    codec_string = chr(codec & 0xFF) + chr((codec >> 8) & 0xFF) + chr((codec >> 16) & 0xFF) + chr((codec >> 24) & 0xFF)

    segment_length_sec = segment_length_min * 60  # Segment length in seconds

    if segment_length_sec > total_duration:
        raise ValueError("Segment length is longer than the video duration")

    # Calculate random start frame
    start_frame = random.randint(0, total_frames - int(segment_length_sec * fps))
    start_time = start_frame / fps  # Start time in seconds

    # Set the starting frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Prepare to save the segment
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output_segment.mp4', fourcc, fps, (int(width), int(height)))

    # Read and write frames
    for _ in range(int(segment_length_sec * fps)):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

    # Create a DataFrame with metadata
    metadata = pd.DataFrame({
        'FPS': [fps],
        'Total Frames': [total_frames],
        'Total Duration (s)': [total_duration],
        'Height (pixels)': [height],
        'Width (pixels)': [width],
        'Codec': [codec_string],
        'Random Start Time (s)': [start_time]
    })

    return metadata

# Example usage
video_path = "resources/Westminster_Hall_09_02_23_13_32_04.mp4"
metadata_df = extract_random_segment(video_path)
print(metadata_df)


    FPS  Total Frames  Total Duration (s)  Height (pixels)  Width (pixels)  \
0  25.0        235728             9429.12            576.0          1024.0   

  Codec  Random Start Time (s)  
0  avc1                7756.44  


In [3]:
from moviepy.editor import VideoFileClip
import random
import pandas as pd
from pydub import AudioSegment

def extract_random_segment(video_path, segment_length_min=10):
    # Load the video
    video = VideoFileClip(video_path)

    # Get total duration of the video in seconds
    total_duration = video.duration

    # Segment length in seconds
    segment_length_sec = segment_length_min * 60

    # Ensure the segment length is not longer than the video
    if segment_length_sec > total_duration:
        raise ValueError("Segment length is longer than the video duration")

    # Calculate random start time in seconds
    start_time = random.uniform(0, total_duration - segment_length_sec)

    # Extract the segment
    segment = video.subclip(start_time, start_time + segment_length_sec)

    # Save the video segment without audio
    segment_video_path = "output_segment.mp4"
    segment.without_audio().write_videofile(segment_video_path, codec="libx264")

    # Save audio as WAV
    audio_path = 'extracted_audio.wav'
    try:
        segment.audio.write_audiofile(audio_path, codec="pcm_s16le")
    except Exception as e:
        print(f"Error in audio extraction: {e}")
        return None

    # Use pydub to read the audio and extract additional metadata
    audio = AudioSegment.from_file(audio_path)
    sampling_rate = audio.frame_rate
    resolution = audio.sample_width * 8  # Sample width in bytes multiplied by 8 to convert to bits

    # Create a DataFrame with metadata
    metadata = pd.DataFrame({
        'FPS': [video.fps],
        'Total Duration (s)': [total_duration],
        'Height (pixels)': [video.size[1]],
        'Width (pixels)': [video.size[0]],
        'Random Start Time (s)': [start_time],
        'Audio Sampling Rate (Hz)': [sampling_rate],
        'Audio Resolution (bits)': [resolution]
    })

    return metadata

# Example usage
video_path = "resources/Westminster_Hall_09_02_23_13_32_04.mp4"
metadata_df = extract_random_segment(video_path)
if metadata_df is not None:
    print(metadata_df)


t:   0%|          | 11/15000 [02:38<59:58:00, 14.40s/it, now=None]

Moviepy - Building video output_segment.mp4.
Moviepy - Writing video output_segment.mp4



t:   0%|          | 11/15000 [07:49<177:32:22, 42.64s/it, now=None]

Moviepy - Done !
Moviepy - video ready output_segment.mp4
MoviePy - Writing audio in extracted_audio.wav


t:   0%|          | 11/15000 [07:58<180:56:43, 43.46s/it, now=None]

MoviePy - Done.
    FPS  Total Duration (s)  Height (pixels)  Width (pixels)  \
0  25.0             9429.12              576            1024   

   Random Start Time (s)  Audio Sampling Rate (Hz)  Audio Resolution (bits)  
0            4031.428606                     44100                       16  


In [4]:
import subprocess
import json

def extract_metadata_ffprobe(video_path):
    cmd = ['ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', '-show_streams', video_path]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return json.loads(result.stdout)

video_path = 'resources/Westminster_Hall_09_02_23_13_32_04.mp4'
metadata = extract_metadata_ffprobe(video_path)
print(json.dumps(metadata, indent=4))

{
    "streams": [
        {
            "index": 0,
            "codec_name": "aac",
            "codec_long_name": "AAC (Advanced Audio Coding)",
            "profile": "LC",
            "codec_type": "audio",
            "codec_tag_string": "mp4a",
            "codec_tag": "0x6134706d",
            "sample_fmt": "fltp",
            "sample_rate": "48000",
            "channels": 2,
            "channel_layout": "stereo",
            "bits_per_sample": 0,
            "id": "0x1",
            "r_frame_rate": "0/0",
            "avg_frame_rate": "0/0",
            "time_base": "1/48000",
            "start_pts": 0,
            "start_time": "0.000000",
            "duration_ts": 452597760,
            "duration": "9429.120000",
            "bit_rate": "61495",
            "nb_frames": "441990",
            "extradata_size": 2,
            "disposition": {
                "default": 1,
                "dub": 0,
                "original": 0,
                "comment": 0,
               

In [8]:
import scipy.signal as signal
import numpy as np
import wave

def butter_bandpass_filter(data, locut, hicut, fs, order):
    nyq = 0.5 * fs
    low = locut / nyq
    high = hicut / nyq
    sos = signal.butter(order, [low, high], analog=False, btype='band', output='sos')
    return signal.sosfilt(sos, data)

def read_wave_file(filename):
    with wave.open(filename, 'r') as w:
        framerate = w.getframerate()
        nframes = w.getnframes()
        data = w.readframes(nframes)
    return np.frombuffer(data, dtype=np.int16), framerate

def write_wave_file(filename, data, framerate):
    with wave.open(filename, 'w') as w:
        w.setnchannels(1)
        w.setsampwidth(2)
        w.setframerate(framerate)
        w.writeframes(data.tobytes())

def process_audio_file(input_file, output_file, order=6):
    # Read the audio file
    data, fs = read_wave_file(input_file)

    # Filter out 48-52 Hz and 96-104 Hz
    filtered_data = butter_bandpass_filter(data, 48, 52, fs, order)
    filtered_data += butter_bandpass_filter(data, 96, 104, fs, order)

    # Write the filtered data to the output file
    write_wave_file(output_file, filtered_data, fs)

# Example usage
input_file = 'extracted_audio.wav'
output_file = 'filtered_audio.wav'
process_audio_file(input_file, output_file)



In [9]:
import wave
import os
import csv
import pathlib

import requests
import numpy as np
import matplotlib.pyplot as plt

from scipy import signal

def load_wav(fpath):
    """Loads a .wav file and returns the data and sample rate.

    :param fpath: the path to load the file from
    :returns: a tuple of (wav file data as a list of amplitudes, sample rate)
    """
    with wave.open(fpath) as wav_f:
        wav_buf = wav_f.readframes(wav_f.getnframes())
        data = np.frombuffer(wav_buf, dtype=np.int16)
        fs = wav_f.getframerate()

        clip_len_s = len(data) / fs
        print(f"Loaded .wav file, n_samples={len(data)} len_s={clip_len_s}")

        return (data, fs)
    
def butter_bandpass_filter(data, locut, hicut, fs, order):
    """Passes input data through a Butterworth bandpass filter. Code borrowed from
    https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html

    :param data: list of signal sample amplitudes
    :param locut: frequency (in Hz) to start the band at
    :param hicut: frequency (in Hz) to end the band at
    :param fs: the sample rate
    :param order: the filter order
    :returns: list of signal sample amplitudes after filtering
    """
    nyq = 0.5 * fs
    low = locut / nyq
    high = hicut / nyq
    sos = signal.butter(order, [low, high], analog=False, btype='band', output='sos')

    return signal.sosfilt(sos, data)

def stft(data, fs):
    """Performs a Short-time Fourier Transform (STFT) on input data.

    :param data: list of signal sample amplitudes
    :param fs: the sample rate
    :returns: tuple of (array of sample frequencies, array of segment times, STFT of input).
        This is the same return format as scipy's stft function.
    """
    window_size_seconds = 16
    nperseg = fs * window_size_seconds
    noverlap = fs * (window_size_seconds - 1)
    f, t, Zxx = signal.stft(data, fs, nperseg=nperseg, noverlap=noverlap)
    return f, t, Zxx

def enf_series(data, fs, nominal_freq, freq_band_size, harmonic_n=1):
    """Extracts a series of ENF values from `data`, one per second.

    :param data: list of signal sample amplitudes
    :param fs: the sample rate
    :param nominal_freq: the nominal ENF (in Hz) to look near
    :param freq_band_size: the size of the band around the nominal value in which to look for the ENF
    :param harmonic_n: the harmonic number to look for
    :returns: a list of ENF values, one per second
    """
    # downsampled_data, downsampled_fs = downsample(data, fs, 300)
    downsampled_data, downsampled_fs = (data, fs)

    locut = harmonic_n * (nominal_freq - freq_band_size)
    hicut = harmonic_n * (nominal_freq + freq_band_size)

    filtered_data = butter_bandpass_filter(downsampled_data, locut, hicut, downsampled_fs, order=10)

    def quadratic_interpolation(data, max_idx, bin_size):
        """
        https://ccrma.stanford.edu/~jos/sasp/Quadratic_Interpolation_Spectral_Peaks.html
        """
        left = data[max_idx-1]
        center = data[max_idx]
        # Your code here
        # ...
        right = data[max_idx+1]

        p = 0.5 * (left - right) / (left - 2*center + right)
        interpolated = (max_idx + p) * bin_size

        return interpolated

    bin_size = f[1] - f[0]

    max_freqs = []
    for spectrum in np.abs(np.transpose(Zxx)):
        max_amp = np.amax(spectrum)
        max_freq_idx = np.where(spectrum == max_amp)[0][0]

        max_freq = quadratic_interpolation(spectrum, max_freq_idx, bin_size)
        max_freqs.append(max_freq)

    return {
        'downsample': {
            'new_fs': downsampled_fs,
        },
        'filter': {
            'locut': locut,
            'hicut': hicut,
        },
        'stft': {
            'f': f,
            't': t,
            'Zxx': Zxx,
        },
        'enf': [f/float(harmonic_n) for f in max_freqs],
    }
    
def pmcc(x, y):
    """Calculates the PMCC between x and y data points.

    :param x: list of x values
    :param y: list of y values, same length as x
    :returns: PMCC of x and y, as a float
    """
    return np.corrcoef(x, y)[0][1]  

def sorted_pmccs(target, references):
    """Calculates and sorts PMCCs between `target` and each of `references`.

    :param target: list of target data points
    :param references: list of lists of reference data points
    :returns: list of tuples of (reference index, PMCC), sorted desc by PMCC
    """
    pmccs = [pmcc(target, r) for r in references]
    sorted_pmccs = [(idx, v) for idx, v in sorted(enumerate(pmccs), key=lambda item: -item[1])]

    return sorted_pmccs

def search(target_enf, reference_enf):
    """Calculates PMCCs between `target_enf` and each window in `reference_enf`.

    :param target_enf: list of target's ENF values
    :param reference_enf: list of reference's ENF values
    :returns: list of tuples of (reference index, PMCC), sorted desc by PMCC
    """
    n_steps = len(reference_enf) - len(target_enf)
    reference_enfs = (reference_enf[step:step+len(target_enf)] for step in range(n_steps))

    coeffs = sorted_pmccs(target_enf, reference_enfs)
    return coeffs

def gb_reference_data(year, month, day=None):
    """Fetches reference ENF data from Great Britain for the given date. Caches responses locally.
    Not used by the example, but included for reference.

    :param year:
    :param month:
    :param day: the day to filter down to. If not provided then entire month is returned
    :returns: list of ENF values
    """
    cache_dir = "./cache/gb"
    pathlib.Path(cache_dir).mkdir(parents=True, exist_ok=True)

    cache_fpath = os.path.join(cache_dir, f"./{year}-{month}.csv")

    if not os.path.exists(cache_fpath):
        ret = requests.get("https://data.nationalgrideso.com/system/system-frequency-data/datapackage.json")
        ret.raise_for_status()

        ret_data = ret.json()
        csv_resource = next(r for r in ret_data['resources'] if r['path'].endswith(f"/fnew-{year}-{month}.csv"))

        ret = requests.get(csv_resource['path'])
        ret.raise_for_status()

        with open(cache_fpath, 'w') as f:
            f.write(ret.text)

    with open(cache_fpath) as f:
        reader = csv.DictReader(f)

        month_data = [(l['dtm'], float(l['f'])) for l in reader]

    if day:
        formatted_date = f"{year}-{str(month).zfill(2)}-{str(day).zfill(2)}"
        return [l for l in month_data if l[0].startswith(formatted_date)]
    else:
        return month_data
    
def plot_stft_ax(ax, f, t, zxx, loclip_f=None, hiclip_f=None):
    """Plots STFT output on the given ax.
    """
    bin_size = f[1] - f[0]
    lindex = int((loclip_f) / bin_size) if loclip_f is not None else 0
    hindex = int((hiclip_f) / bin_size) if hiclip_f is not None else -1

    ax.pcolormesh(t, f[lindex:hindex], np.abs(zxx[lindex:hindex]), shading='gouraud')


def plot_series_ax(ax, series, label=None):
    """Plots a series on the given ax.
    """
    t = np.linspace(0, len(series), num=len(series))
    ax.plot(t, series, label=label)
    
if __name__ == "__main__":
    nominal_freq = 50
    freq_band_size = 0.2

    ref_data, ref_fs = load_wav("extracted_audio.wav")

    ref_enf_output = enf_series(ref_data, ref_fs, nominal_freq, freq_band_size)
    ref_enf = ref_enf_output['enf']

    # !!!: make sure to run ./bin/download-example-files first
    data, fs = load_wav("./001.wav")

    harmonic_n = 1
    enf_output = enf_series(data, fs, nominal_freq, freq_band_size, harmonic_n=2)
    target_enf = enf_output['enf']

    stft = enf_output['stft']
    f = stft['f']
    t = stft['t']
    Zxx = stft['Zxx']

    pmccs = search(target_enf, ref_enf)
    print(pmccs[0:100])
    predicted_ts = pmccs[0][0]
    print(f"Best predicted timestamp is {predicted_ts}")
    # True value provided by creator of example file
    print("True value is 71458")

    filt = enf_output['filter']
    locut = filt['locut']
    hicut = filt['hicut']

    # Plot the target ENF and the matched reference section on the same axes
    fig, ax = plt.subplots(1)
    plt.title("Target and matched reference section ENFs")
    plot_series_ax(ax, target_enf, "target")
    plot_series_ax(ax, ref_enf[predicted_ts:predicted_ts+len(target_enf)], "ref")
    ax.legend()
    plt.show()

    # Plot the target's frequency spectrum around 50Hz
    fig, ax = plt.subplots(1)
    plt.title("Target frequency spectra over time")
    plot_stft_ax(ax, f, t, Zxx, loclip_f=locut-0.5, hiclip_f=hicut+0.5)
    plt.show()
## audio_data, fs = load_wav('filtered_audio.wav')
## data_filtered = butter_bandpass_filter(audio_data, 48, 52, fs, 6)


The TDMF function returns Output, which is the smoothed version of the Input signal. Spikes or outliers in the Input signal are replaced with median values, making the Output signal smoother and more consistent, ideal for further analysis or processing.

This function is particularly useful in scenarios where maintaining the general trend of the data is important, but sharp, isolated fluctuations (which could be noise or errors) need to be removed.

In [ ]:
import math

def temporal_discrete_median_filter(input_signal, window_order, threshold):
    """
    Apply Temporal Discrete Median Filtering to a signal.

    Parameters:
    input_signal (list): The original signal to be smoothed.
    window_order (int): The size of the rolling window for median calculation.
    threshold (float): The threshold value to determine outliers.

    Returns:
    list: The smoothed signal after applying TDMF.
    """

    # Length of the input signal
    input_length = len(input_signal)

    # Initialize arrays to store median-filtered and final output values
    median_filtered_signal = [None] * input_length
    output_signal = [None] * input_length

    # Calculate padding length for each side of the signal
    padding_length = int((window_order - 1) / 2)

    # Create padding arrays for the start and end of the signal
    start_padding = [input_signal[0]] * padding_length
    end_padding = [input_signal[-1]] * padding_length

    # Combine padding and original signal into one padded signal
    padded_signal = start_padding + input_signal + end_padding

    # Calculate median for each window position
    for i in range(input_length):
        window = padded_signal[i: i + window_order]
        window.sort()
        median_filtered_signal[i] = window[math.ceil((window_order - 1) / 2)]

    # Calculate the difference between the median-filtered signal and the original signal
    detrended_signal = [median_filtered_signal[i] - input_signal[i] for i in range(input_length)]

    # Apply the threshold to determine whether to use the original or median value
    for i in range(input_length):
        if abs(detrended_signal[i]) <= threshold:
            output_signal[i] = input_signal[i]
        else:
            output_signal[i] = median_filtered_signal[i]

    return output_signal


In [11]:
"""
This script processes an audio file to perform a Short-Time Fourier Transform (STFT) in a memory-efficient way. 
STFT is a method to analyze the frequency content of audio signals over time. However, STFT can be memory-intensive 
for long audio files or large window sizes. This script addresses this issue by processing the audio file in smaller 
chunks, reducing the overall memory requirement.

Functions:
- read_wave_file: Reads a WAV file and returns its data and sample rate.
- stft: Performs STFT on audio data in chunks to manage memory usage.
- process_stft: Main function to read an audio file and process it using STFT.
"""

import scipy.signal as signal
import numpy as np
import wave

def read_wave_file(filename):
    """
    Read a WAV audio file.

    Parameters:
    filename (str): The path to the WAV file.

    Returns:
    tuple: A tuple containing two elements:
           - A NumPy array with the audio data.
           - The sample rate (framerate) of the audio file.
    """
    with wave.open(filename, 'r') as w:
        framerate = w.getframerate()
        nframes = w.getnframes()
        data = w.readframes(nframes)
    return np.frombuffer(data, dtype=np.int16), framerate

def stft(data, fs, chunk_size=1024*1024):
    """
    Perform Short-Time Fourier Transform on audio data in chunks.

    Parameters:
    data (np.array): The audio data as a NumPy array.
    fs (int): The sample rate of the audio data.
    chunk_size (int): The size of each chunk for processing. Default is 1 MB.

    Returns:
    tuple: A tuple containing three elements:
           - An array of frequencies.
           - An array of time segments.
           - The STFT matrix (complex values).
    """
    window_size_seconds = 16
    nperseg = fs * window_size_seconds
    noverlap = fs * (window_size_seconds - 1)
    frequencies, times, Zxx = None, None, None

    for i in range(0, len(data), chunk_size):
        chunk_data = data[i:i+chunk_size]
        f, t, zxx_chunk = signal.stft(chunk_data, fs, nperseg=nperseg, noverlap=noverlap)

        if Zxx is None:
            frequencies, times, Zxx = f, t, zxx_chunk
        else:
            Zxx = np.hstack((Zxx, zxx_chunk))

    return frequencies, times, Zxx

def process_stft(input_file):
    """
    Process an audio file using STFT.

    Parameters:
    input_file (str): Path to the input WAV file.

    Returns:
    tuple: A tuple containing three elements:
           - An array of frequencies.
           - An array of time segments.
           - The STFT matrix (complex values).
    """
    data, fs = read_wave_file(input_file)
    f, t, Zxx = stft(data, fs)
    return f, t, Zxx

# Example usage
input_file = 'filtered_audio.wav'
frequencies, times, stft_matrix = process_stft(input_file)


In [6]:
def enf_series(data, fs, nominal_freq, freq_band_size, harmonic_n=1):
    """Extracts a series of ENF values from `data`, one per second.

    :param data: list of signal sample amplitudes
    :param fs: the sample rate
    :param nominal_freq: the nominal ENF (in Hz) to look near
    :param freq_band_size: the size of the band around the nominal value in which to look for the ENF
    :param harmonic_n: the harmonic number to look for
    :returns: a list of ENF values, one per second
    """
    # downsampled_data, downsampled_fs = downsample(data, fs, 300)
    downsampled_data, downsampled_fs = (data, fs)

    locut = harmonic_n * (nominal_freq - freq_band_size)
    hicut = harmonic_n * (nominal_freq + freq_band_size)

    filtered_data = butter_bandpass_filter(downsampled_data, locut, hicut, downsampled_fs, order=10)

    f, t, Zxx = stft(filtered_data, downsampled_fs)

    def quadratic_interpolation(data, max_idx, bin_size):
        """
        https://ccrma.stanford.edu/~jos/sasp/Quadratic_Interpolation_Spectral_Peaks.html
        """
        left = data[max_idx-1]
        center = data[max_idx]
        right = data[max_idx+1]

        p = 0.5 * (left - right) / (left - 2*center + right)
        interpolated = (max_idx + p) * bin_size

        return interpolated

    bin_size = f[1] - f[0]

    max_freqs = []
    for spectrum in np.abs(np.transpose(Zxx)):
        max_amp = np.amax(spectrum)
        max_freq_idx = np.where(spectrum == max_amp)[0][0]

        max_freq = quadratic_interpolation(spectrum, max_freq_idx, bin_size)
        max_freqs.append(max_freq)

    return {
        'downsample': {
            'new_fs': downsampled_fs,
        },
        'filter': {
            'locut': locut,
            'hicut': hicut,
        },
        'stft': {
            'f': f,
            't': t,
            'Zxx': Zxx,
        },
        'enf': [f/float(harmonic_n) for f in max_freqs],
    }
